<a href="https://colab.research.google.com/github/DutchVandaline/Artificial_Intelligence/blob/main/GPT2_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building GPT2 from scratch.

Building GPT2 is a complex task but I guess I can learn about transformers. For now on, I don't really understand about `attention` and no basics with `RNN`, `LSTM`. It might be a challenging task to understand the transformers of course. But, I have built ViT (Vision Transformer). Why don't I give it a shot?

* References

  * Illustrated-gpt2 by jay alammar : https://jalammar.github.io/illustrated-gpt2/#part-1-got-and-language-modeling
  
  * Here's how you can build and train GPT-2 : https://dev.to/amit_kharel_aae65abe2b111/heres-how-you-can-build-and-train-gpt-2-from-scratch-using-pytorch-345n

I've used jay alammar's blog post to understand the architecture and how gpt2 works and bottom blog post is for dataset and preprocessing.

In [2]:
import torch
import torch.nn
from torch import nn

import matplotlib.pyplot as plt
print(f"torch version: {torch.__version__}")

torch version: 2.4.0+cpu


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### Building Character based Encoder and Decoder

In [10]:
data_dir = "data.txt"
text = open(data_dir, 'r').read()

chars = list(set(text))
vocab_size = len(chars)

print(f"We have total {vocab_size} vocabularies and they are\n {''.join(chars)}")

We have total 89 vocabularies and they are
 9!Xj0;R2W)1KóïOrkpT
oíMéP”—'“LEctx4"hD5bJf8imy.I/?6(e:nzlq,s[adZ&w3B*\VHU] GuASF7N-Y’vgCQ


In [22]:
chr_to_idx = {c:i for i, c in enumerate(chars)}
idx_to_chr = {i:c for i, c in enumerate(chars)}

def encode(input_text: str) -> list[int]:
    return [chr_to_idx[t] for t in input_text]

def decode(input_tokens: list[int]) -> str:
    return "".join([idx_to_chr[i] for i in input_tokens])

encoded_word = encode("Hello World")
print(f"Hello World -> {encoded_word}")

Hello World -> [71, 52, 56, 56, 20, 74, 8, 20, 15, 56, 62]


In [18]:
data = torch.tensor(encode(text), dtype=torch.long, device=device)
data

tensor([71, 52, 74,  ..., 56, 52, 35])